In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import random
import pandas as pd
import seaborn as sns
import re

In [2]:
def create_vgg16_model():
    conv_base = VGG16(weights='imagenet',
                      include_top=False,
                      input_shape=(150, 150, 3))
    
    for layer in conv_base.layers[:-4]:
        layer.trainable = False
    
    model = models.Sequential([
        conv_base,
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(loss='binary_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  metrics=['accuracy'])
    return model

## 3. Architecture du Modèle - VGG16 avec Fine-Tuning

### 🧬 Principe du Transfer Learning

Le **transfer learning** consiste à utiliser un modèle pré-entraîné (ici VGG16) et l'adapter à notre tâche spécifique. Cette approche présente plusieurs avantages :

- ⚡ **Accélération de l'entraînement** : Utilisation de features déjà apprises
- 📈 **Amélioration des performances** : Bénéfice de l'expertise acquise sur ImageNet
- 💾 **Réduction des données nécessaires** : Efficace même avec un dataset limité
- 🎯 **Meilleure généralisation** : Features pré-entraînées robustes

### 🏗️ Architecture du modèle VGG16

**VGG16** est un réseau de neurones convolutionnel composé de :
- **13 couches convolutionnelles** avec filtres 3×3
- **5 couches de pooling** pour réduire la dimensionnalité  
- **3 couches fully connected** (exclues avec `include_top=False`)

### ⚙️ Configuration du Fine-Tuning

#### Chargement du modèle pré-entraîné
```python
conv_base = VGG16(weights='imagenet',           # Poids pré-entraînés sur ImageNet
                  include_top=False,            # Exclusion des couches de classification
                  input_shape=(150, 150, 3))    # Adaptation à nos images 150×150×3
```

#### Stratégie de gel des couches
```python
for layer in conv_base.layers[:-4]:
    layer.trainable = False
```

**Principe du fine-tuning :**
- 🔒 **Couches gelées** : Les premières couches (features de bas niveau) restent figées
- 🔓 **Couches dégelées** : Les 4 dernières couches sont réentraînées pour notre tâche
- 🎯 **Objectif** : Adapter les features de haut niveau aux radiographies thoraciques

### 🏛️ Architecture finale du modèle

#### Couches ajoutées pour la classification
1. **`Flatten()`** : Conversion des feature maps 2D en vecteur 1D
2. **`Dense(256, activation='relu')`** : Couche fully connected avec 256 neurones
3. **`Dropout(0.5)`** : Régularisation pour éviter le surapprentissage (50% dropout)
4. **`Dense(1, activation='sigmoid')`** : Couche de sortie pour classification binaire

#### Configuration de l'optimisation
- **Loss function** : `binary_crossentropy` (adapté à la classification binaire)
- **Optimizer** : `Adam` avec learning rate faible (0.0001)
- **Métriques** : `accuracy` pour suivre les performances

### 💡 Avantages de cette architecture

|           Aspect          |                       Bénéfice                            |
|---------------------------|-----------------------------------------------------------|
| **Poids pré-entraînés**   | Connaissance préalable des motifs visuels                 |
| **Fine-tuning sélectif**  | Adaptation spécifique sans perdre les features générales  |
| **Dropout**               | Prévention du surapprentissage                            |
| **Learning rate faible**  | Ajustements fins sans perturber les features pré-apprises |

> ⚠️ **Conseil** : Cette architecture équilibre performance et efficacité computationnelle, particulièrement adaptée à la classification d'images médicales avec des datasets de taille modérée.